# Create and Deploy the Thumbnail Generator



## Create the Template Application

Load environment variables:

In [ ]:
. demo.env

Create the Oracle Function Application directory:

In [ ]:
mkdir -p build/$APP_NAME && cd build/$APP_NAME

Initialise the Fn project:

In [ ]:
fn init --runtime python $FN_NAME

Create the application using the subnet specified by the environment variable `SUBNET_OCID`:

In [ ]:
fn create app $APP_NAME --annotation oracle.com/oci/subnetIds=[\"$SUBNET_OCID\"]

Deploy the function:

In [ ]:
cd $FN_NAME && \
fn deploy -v --app $APP_NAME

Invoke the function:

In [ ]:
fn invoke $APP_NAME $FN_NAME

## OCI Logging

> To allow Functions developers to manage the logging in the compartment, add the following statements to the *Fn-Demo-Policy*:
> ```
> Allow group FunctionsDevelopers to use log-groups in compartment DEMO
> Allow group FunctionsDevelopers to manage log-groups in compartment DEMO
> Allow group FunctionsDevelopers to read log-groups in compartment DEMO
> Allow group FunctionsDevelopers to read log-content in compartment DEMO
> ``` 

Enable logging for the Oracle Function, create the log group `Demo_Log_Group`.
Reinvoke the function and review the OCI logs.

## Update the Application to Generate Thumbnails

> **Note**. The same code contains a custom `Dockerfile` as we will be using the Python package [*python-magic"](https://pypi.org/project/python-magic/) that is used to obtain the MIME type of the image file. This package has a dependency on the *libmagic* C library and will need to be installed.

Copy the sample code into the Fn project directory:

In [ ]:
cd $PROJECT_ROOT && \
cp -r sample-code/$APP_NAME/$FN_NAME/* build/$APP_NAME/$FN_NAME/

Configure the application's parameters:

In [ ]:
fn config app $APP_NAME output_bucket thumbnails && \
fn config app $APP_NAME thumbnail_size_pixels 256

Re-deploy the function:

In [ ]:
cd $PROJECT_ROOT/build/$APP_NAME/$FN_NAME && \
fn deploy -v --app $APP_NAME

## Final Demonstration with Events

> To allow Functions developers to manage the event rules in the compartment, add the following statements to the *Fn-Demo-Policy*:
> ```
> Allow group FunctionsDevelopers to manage cloudevents-rules in compartment DEMO
> Allow group FunctionsDevelopers to inspect compartments in compartment DEMO
> Allow group FunctionsDevelopers to use tag-namespaces in compartment DEMO
> Allow group FunctionsDevelopers to use virtual-network-family in compartment DEMO
> Allow group FunctionsDevelopers to manage function-family in compartment DEMO
> ```

1. Create the Event rule `CloudEventRule_GenerateThumbail` in compartment `DEMO`.
1. Select the service name *Object Storage*.
1. Chose the following event types:
    * `Object - Create`
    * `Object - Update`
    * `Object - Delete`
1. Add a condition *Attribute* using the attribute name *bucketName*. Enter the value `photos`.
1. Add an action of type *Functions*. Select the compartment, application and function.
1. Click the *Create Rule* button to conclude. 
1. Once the event has been created, enable logging to assist in troubleshooting.